# 2.Improving Deep Neural Networks:Hyperparameter Tuning,Regularization and Optimization

## 2.3Gradient Checking

In [8]:
import numpy as np

In [9]:
def forward_propagation(x, theta):
    ### START CODE HERE ### (approx. 1 line)
    J = theta * x
    ### END CODE HERE ###
    return J

In [10]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

J = 8


In [11]:
# GRADED FUNCTION: backward_propagation
def backward_propagation(x, theta):
    """
    Computes the derivative of J with respect to theta (see Figure).
    Arguments:
    x -- a real-valued input
    theta -- our parameter, a real number as well
    Returns:
    dtheta -- the gradient of the cost with respect to theta
    """
    ### START CODE HERE ### (approx. 1 line)
    dtheta = x
    ### END CODE HERE ###
    return dtheta

In [12]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 2


In [15]:
def gradient_check(x, theta, epsilon = 1e-7):
    thetaplus = theta + epsilon # Step 1
    thetaminus = theta - epsilon # Step 2
    J_plus = thetaplus * x # Step 3
    J_minus = thetaminus * x # Step 4
    gradapprox = ( J_plus-J_minus)/(2*epsilon) # Step 5
    grad = backward_propagation(x, theta)
    ### END CODE HERE ###
    ### START CODE HERE ### (approx. 1 line)
    numerator = np.linalg.norm(grad-gradapprox) 
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox) 
    difference = numerator/denominator 
    ### END CODE HERE ###
    if difference < 1e-7:
        print ("The gradient is correct!")
    else:
        print ("The gradient is wrong!")
    
    return difference

In [16]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

The gradient is correct!
difference = 2.919335883291695e-10


In [17]:
def forward_propagation_n(X, Y, parameters):
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    # Cost
    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    return cost, cache

In [22]:
from testCases import *
from gc_utils import *

In [23]:
def backward_propagation_n(X, Y, cache):
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) * 2
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 4./m * np.sum(dZ1, axis=1, keepdims = True)
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,"dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    return gradients

In [24]:
def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7):
    parameters_values, _ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    for i in range(num_parameters):    
        thetaplus = np.copy(parameters_values) # Step 1
        thetaplus[i][0] = thetaplus[i][0] + epsilon # Step 2
        J_plus[i], _ = forward_propagation_n(X, Y,vector_to_dictionary(thetaplus)) # Step 3
        ### END CODE HERE ###

        thetaminus = np.copy(parameters_values) # Step 1
        thetaminus[i][0] = thetaminus[i][0] - epsilon # Step 2
        J_minus[i], _ = forward_propagation_n(X, Y,vector_to_dictionary(thetaminus)) # Step 3
        ### END CODE HERE ###
        # Compute gradapprox[i]
        ### START CODE HERE ### (approx. 1 line)
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2.* epsilon)
    ### END CODE HERE ###
    # Compare gradapprox to backward propagation gradients by computing difference.
    ### START CODE HERE ### (approx. 1 line)
    numerator = np.linalg.norm(grad - gradapprox) # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox) #
    difference = numerator / denominator # Step 3'
    ### END CODE HERE ###
    if difference > 1e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    return difference

In [25]:
X, Y, parameters = gradient_check_n_test_case()
cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

There is a mistake in the backward propagation! difference = 0.2850931567761624
